# Human Detection Movement

In [ ]:
# --- IMPORTING LIBRARIES ---
from ultralytics import YOLO
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math
import threading
from traitlets.config.configurable import SingletonConfigurable

import motors # ADDED LIBRARY

def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg',value)[1])

# --- Load the YOLO model ---
model = YOLO("yolo11l_half.engine")
# model = YOLO("yolo11n.engine")

# Initialize Robot Class
robot = motors.MotorsYukon(mecanum=False)

In [ ]:
class Camera(SingletonConfigurable):
    color_value = traitlets.Any()
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()

        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA #VGA(672*376), HD720(1280*720), HD1080 (1920*1080) or ...
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # Use ULTRA depth mode
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Use meter units (for depth measurements)

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()

        self.thread_runnning_flag = False

        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU) 

    def _capture_frames(self):

        while(self.thread_runnning_flag==True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value_BGRA = self.image.get_data()
                self.color_value= cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data()) 

                """
                depth_image_test = self.depth_image.copy()  #copy the depth image
                depth_image_test = np.nan_to_num(depth_image_test, nan=0.0).astype(np.float32)
                depth_image_test[:94,:] = 0
                depth_image_test[282:,:]=0
                depth_image_test[:,:168]=0
                depth_image_test[:,504:]=0

                depth_image_test[depth_image_test<100]=0
                depth_image_test[depth_image_test>1000]=0

                distance = depth_image_test[depth_image_test!=0].min()
                """
                
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag=True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()    
            
camera = Camera()
camera.start()

[2025-03-12 11:03:42 UTC][ZED][INFO] Logging level INFO
[2025-03-12 11:03:42 UTC][ZED][INFO] Logging level INFO
[2025-03-12 11:03:42 UTC][ZED][INFO] Logging level INFO
[2025-03-12 11:03:43 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-03-12 11:03:44 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-03-12 11:03:44 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-03-12 11:03:44 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-03-12 11:03:44 UTC][ZED][INFO] [Init]  Serial Number: S/N 32565960
[2025-03-12 11:03:44 UTC][ZED][WARNING] [Init]  Self-calibration failed. Point the camera towards a more textured and brighter area. Avoid objects closer than 1 meter (Error code: 0x01) 
Loading yolo11l_half.engine for TensorRT inference...


In [ ]:
# --- DISPLAY WIDGETS ---
display_color = widgets.Image(format='jpeg', width='45%') 
display_depth = widgets.Image(format='jpeg', width='45%')  
layout=widgets.Layout(width='100%')
sidebyside = widgets.HBox([display_color, display_depth],layout=layout) 
display(sidebyside)


def func(change):

    frame = change['new']
    # t1 = cv2.getTickCount()
    results = model(frame,verbose=False)
    # total_time = (cv2.getTickCount() - t1) / cv2.getTickFrequency()
    # print('total_time ',total_time)

    # --- COLLISION ALGORITHM [Might need to play around with location]  ---
    """
    depth_image = camera.depth_image
    if depth_image is None or depth_image.size == 0:
        print("Invalid depth image")
        return
    
    # Clean up the depth image (remove NaN and inf values)
    depth_image_clean = np.nan_to_num(depth_image, nan=0.0)  # Replace NaN values with 0.0
    depth_image_clean[depth_image_clean < 100] = 0  # Filter out close objects (under 100mm)
    depth_image_clean[depth_image_clean > 1000] = 0  # Filter out far objects (over 1000mm)
    
    # Get the minimum valid depth in the frame (ignoring zeroes)
    valid_depth_pixels = depth_image_clean[depth_image_clean != 0]
    
    if valid_depth_pixels.size == 0:
        print("No valid depth pixels detected")
        return

    distance = valid_depth_pixels.min()  # Get the minimum valid distance
    """
    
    conf_threshold = 0.8
    frame_center = frame.shape[1] // 2 # GET HORIZONTAL CENTER OF FRAME
    if len(results) == 0:
        robot.stop()
    for result in results:
        if 0 in result.boxes.cls:
            for i in range (len(result.boxes.cls)):
                if(result.boxes.cls[i] == 0):
                    if (result.boxes.conf[i] > conf_threshold):
                        bbox = result.boxes.xyxy[i]
                        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
                        
                        # -- MOVEMENT AFTER DETECTING OBJECT ---     
                        # --- MOVEMENT WITH CENTER READJUSTMENT ---
                        obj_center = int((bbox[0] + bbox[2]) / 2) # GETS OBJECT CENTER
                        cv2.circle(frame, (obj_center, int((bbox[1] + bbox[3]) / 2)), 5, (0, 255, 0), -1)
                        
                        if abs(obj_center - frame_center) > 50:
                            if obj_center < frame_center:
                                robot.left(0.3)
                            else:
                                robot.right(0.3)
                        else:
                            robot.forward(0.5)   
        else:
            robot.stop()
                    
    scale = 0.1 
    resized_image = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_image)

    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(camera.depth_image , alpha=0.03), cv2.COLORMAP_JET)
    resized_depth_colormap=cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_depth.value = bgr8_to_jpeg(resized_depth_colormap)

    # --- ADDED CENTER CIRCLE ---
    cv2.circle(resized_image, (int(camera.width*scale//2),int(camera.height*scale//2)), 1, (0, 255, 0))
    # --- May need to place center circle code in _capture_frames function ---


camera.observe(func, names=['color_value'])
# camera.stop()

[03/12/2025-11:03:45] [TRT] [I] Loaded engine size: 52 MiB
[03/12/2025-11:03:45] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[03/12/2025-11:03:45] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +1, GPU +36, now: CPU 1, GPU 84 (MiB)


In [ ]:
camera.stop()
robot.stop()